# Character based RNN language model trained on 'The Complete Works of William Shakespeare'
Based on http://karpathy.github.io/2015/05/21/rnn-effectiveness

In [1]:
using Pkg; for p in ("Knet","AutoGrad"); haskey(Pkg.installed(),p) || Pkg.add(p); end

In [2]:
RNNTYPE = :lstm
BATCHSIZE = 256
SEQLENGTH = 100
INPUTSIZE = 168
VOCABSIZE = 84
HIDDENSIZE = 334
NUMLAYERS = 1
DROPOUT = 0.0
LR=0.001
BETA_1=0.9
BETA_2=0.999
EPS=1e-08
EPOCHS = 30;

In [3]:
# Load 'The Complete Works of William Shakespeare'
using Knet; @show gpu()
include(Knet.dir("data","gutenberg.jl"))
trn,tst,chars = shakespeare()
map(summary,(trn,tst,chars))

gpu() = 0


("4934845-element Array{UInt8,1}", "526731-element Array{UInt8,1}", "84-element Array{Char,1}")

In [4]:
# Print a sample
println(string(chars[trn[1020:1210]]...)) 


    Cheated of feature by dissembling nature,
    Deform'd, unfinish'd, sent before my time
    Into this breathing world scarce half made up,
    And that so lamely and unfashionable
 


In [5]:
# Minibatch data
function mb(a)
    N = div(length(a),BATCHSIZE)
    x = reshape(a[1:N*BATCHSIZE],N,BATCHSIZE)' # reshape full data to (B,N) with contiguous rows
    minibatch(x[:,1:N-1], x[:,2:N], SEQLENGTH) # split into (B,T) blocks 
end
dtrn,dtst = mb(trn),mb(tst)
map(length, (dtrn,dtst))

(192, 20)

In [6]:
# Define model
function initmodel()
    w(d...)=KnetArray(xavier(Float32,d...))
    b(d...)=KnetArray(zeros(Float32,d...))
    r,wr = rnninit(INPUTSIZE,HIDDENSIZE,rnnType=RNNTYPE,numLayers=NUMLAYERS,dropout=DROPOUT)
    wx = w(INPUTSIZE,VOCABSIZE)
    wy = w(VOCABSIZE,HIDDENSIZE)
    by = b(VOCABSIZE,1)
    return r,wr,wx,wy,by
end;

In [7]:
# Define loss and its gradient
function predict(ws,xs,hx,cx;pdrop=0)
    r,wr,wx,wy,by = ws
    x = wx[:,xs]                                    # xs=(B,T) x=(X,B,T)
    x = dropout(x,pdrop)
    y,hy,cy = rnnforw(r,wr,x,hx,cx,hy=true,cy=true) # y=(H,B,T) hy=cy=(H,B,L)
    y = dropout(y,pdrop)
    y2 = reshape(y,size(y,1),size(y,2)*size(y,3))   # y2=(H,B*T)
    return wy*y2.+by, hy, cy
end

function loss(w,x,y,h;o...)
    py,hy,cy = predict(w,x,h...;o...)
    h[1],h[2] = value(hy),value(cy)
    return nll(py,y)
end
using AutoGrad: gradloss
lossgradient = gradloss(loss);

In [8]:
# Train and test loops
function train(model,data,optim)
    hiddens = Any[nothing,nothing]
    Σ,N=0,0
    for (x,y) in data
        grads,loss1 = lossgradient(model,x,y,hiddens;pdrop=DROPOUT)
        update!(model, grads, optim)
        Σ,N=Σ+loss1,N+1
    end
    return Σ/N
end

function test(model,data)
    hiddens = Any[nothing,nothing]
    Σ,N=0,0
    for (x,y) in data
        Σ,N = Σ+loss(model,x,y,hiddens), N+1
    end
    return Σ/N
end; 

In [9]:
# Train model or load from file if exists
model=optim=nothing; 
Knet.gc()
if !isfile("shakespeare.jld2")
    model = initmodel()
    optim = optimizers(model, Adam; lr=LR, beta1=BETA_1, beta2=BETA_2, eps=EPS);    
    @info("Training...")
    @time for epoch in 1:EPOCHS
        @time trnloss = train(model,dtrn,optim) # ~18 seconds
        @time tstloss = test(model,dtst)        # ~0.5 seconds
        println((:epoch, epoch, :trnppl, exp(trnloss), :tstppl, exp(tstloss)))
    end
    Knet.save("shakespeare.jld2","model",model)
else
    model = Knet.load("shakespeare.jld2","model")
end
summary(model)

┌ Info: Training...
└ @ Main In[9]:7
┌ Warning: `getval` is deprecated, use `value` instead.
│   caller = #loss#8(::Base.Iterators.Pairs{Symbol,Float64,Tuple{Symbol},NamedTuple{(:pdrop,),Tuple{Float64}}}, ::Function, ::Param{Tuple{RNN,KnetArray{Float32,3},KnetArray{Float32,2},KnetArray{Float32,2},KnetArray{Float32,2}}}, ::Array{UInt8,2}, ::Array{UInt8,2}, ::Array{Any,1}) at In[7]:14
└ @ Main ./In[7]:14
┌ Warning: `getval` is deprecated, use `value` instead.
│   caller = #loss#8(::Base.Iterators.Pairs{Symbol,Float64,Tuple{Symbol},NamedTuple{(:pdrop,),Tuple{Float64}}}, ::Function, ::Param{Tuple{RNN,KnetArray{Float32,3},KnetArray{Float32,2},KnetArray{Float32,2},KnetArray{Float32,2}}}, ::Array{UInt8,2}, ::Array{UInt8,2}, ::Array{Any,1}) at In[7]:14
└ @ Main ./In[7]:14


 24.699409 seconds (13.49 M allocations: 856.929 MiB, 1.45% gc time)


┌ Warning: `getval` is deprecated, use `value` instead.
│   caller = #loss#8(::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}, ::Function, ::Tuple{RNN,KnetArray{Float32,3},KnetArray{Float32,2},KnetArray{Float32,2},KnetArray{Float32,2}}, ::Array{UInt8,2}, ::Array{UInt8,2}, ::Array{Any,1}) at In[7]:14
└ @ Main ./In[7]:14
┌ Warning: `getval` is deprecated, use `value` instead.
│   caller = #loss#8(::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}, ::Function, ::Tuple{RNN,KnetArray{Float32,3},KnetArray{Float32,2},KnetArray{Float32,2},KnetArray{Float32,2}}, ::Array{UInt8,2}, ::Array{UInt8,2}, ::Array{Any,1}) at In[7]:14
└ @ Main ./In[7]:14


  0.964949 seconds (1.39 M allocations: 83.647 MiB, 2.74% gc time)
(:epoch, 1, :trnppl, 14.17144f0, :tstppl, 8.031225f0)
 18.063935 seconds (275.04 k allocations: 172.994 MiB, 0.15% gc time)
  0.575244 seconds (7.92 k allocations: 13.342 MiB)
(:epoch, 2, :trnppl, 6.8886585f0, :tstppl, 6.1861835f0)
 18.169467 seconds (275.45 k allocations: 172.999 MiB, 0.10% gc time)
  0.582032 seconds (10.64 k allocations: 13.383 MiB, 2.66% gc time)
(:epoch, 3, :trnppl, 5.634745f0, :tstppl, 5.3615026f0)
 18.233755 seconds (277.01 k allocations: 173.022 MiB, 0.12% gc time)
  0.583114 seconds (7.92 k allocations: 13.342 MiB)
(:epoch, 4, :trnppl, 4.9907575f0, :tstppl, 4.882069f0)
 18.256127 seconds (276.47 k allocations: 173.014 MiB, 0.08% gc time)
  0.581267 seconds (10.64 k allocations: 13.383 MiB, 0.31% gc time)
(:epoch, 5, :trnppl, 4.5789294f0, :tstppl, 4.5613937f0)
 18.254528 seconds (277.01 k allocations: 173.022 MiB, 0.13% gc time)
  0.586195 seconds (7.92 k allocations: 13.342 MiB)
(:epoch, 6, :tr

"Tuple{RNN,KnetArray{Float32,3},KnetArray{Float32,2},KnetArray{Float32,2},KnetArray{Float32,2}}"

In [10]:
# Sample from trained model
function generate(model,n)
    function sample(y)
        p,r=Array(exp.(Array(y).-logsumexp(y))),rand()
        for j=1:length(p); (r -= p[j]) < 0 && return j; end
    end
    h,c = nothing,nothing
    x = something(findfirst(isequal('\n'), chars), 0)
    for i=1:n
        y,h,c = predict(model,[x],h,c)
        x = sample(y)
        print(chars[x])
    end
    println()
end

generate(model,1000)


                         Enter a Seins of
                       No. Marcorius Marcious Comanio, there is nothing of your England.

  Nurse. Gods fall, devouring.

         Enter angar

                         Enter MISTRESS QUINDER and to LAVINIA

  BOLINBBRO. I never take a messenger (look-like musicon
    A gar of my good master's true. He may wejt,
    Breathe himself, I risely: they mook'd him,
    And sit on o'er your emports, if you were first of all
    To see thou takes from faith, and fly but Scotlif her!
    But look a My life too, O hoop on quarrels!
    I will confess you to come for thee.
  ROSALIND. Look should all undertake the old body of Cailia
    Again, whence it is spurn.
  NERISSA. You may think on's stame.
  POLICE. Let us so low thyself, and vest so many loss.
  SAFERS. His house?
  SECOND SENATOR. They have itsly accompaty?
  IAGO. I know wild love what he shall civil grief do curl'd not upon me.
                                        
